In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
from langchain.llms import OpenAI, AzureOpenAI

llm = OpenAI(model_name="text-davinci-003")

text = "What would be a good company name for a company that makes colorful socks?:"
llm.predict(text)
"""
> Rainbow Toes Socks
"""

In [ ]:
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI 
from langchain.schema.messages import HumanMessage, SystemMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content= text),
]
chat = ChatOpenAI(model_name="gpt-3.5-turbo-1106")
print(chat.predict(messages))
"""
> Rainbow Toe Co.
"""

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me the names of the most typical meals from: {country}"
)
meals_prompt = prompt_template.format(country="Colombia")
"""
> Tell me the names of the most typical meals from 
"""
llm.predict(meals_prompt)
"""
> 1. Bandeja Paisa, 2. Arepas, 3. Sancocho, 4. Ajiaco, 5. Tamales...
"""

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

messages = chat_prompt.format_messages(input_language="English",
                            output_language="Spanish",
                           text= "Langchain is awesome")
"""
>
"""
messages

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

translation_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106")
#Chain
translation_chain = translation_prompt | llm

#Run chain
translation_params = {"input_language":"English",
                      "output_language":"Spanish",
                      "text":"Langchain is awesome "}
translation_chain.invoke(translation_params)

In [ ]:
translation_chain.invoke(translation_params)

In [ ]:
for s in translation_chain.stream(translation_params):
    print(s.content, end="", flush=True)

In [ ]:
from langchain.schema import BaseOutputParser
class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""
    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a country, and you should generate 5 food of that country in a comma separated list.
ONLY return a comma separated list, and nothing more."""

human_template = "{country}"

meals_prompt = ChatPromptTemplate.from_messages([
    ("system", template), 
    ("human", human_template), 
])

llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106")

meals_chain = meals_prompt | llm | CommaSeparatedListOutputParser()

chain.invoke({"country":"Colombia"})

# >['Arepas', 'Bandeja Paisa', 'Empanadas', 'Ajiaco', 'Lechona']

In [ ]:
meals_chain = meals_prompt | llm

In [ ]:
meals_chain.invoke({"country":"colombia"})

In [ ]:
for s in meals_chain.stream({"country": "Colombia"}):
    print(s.content, end="", flush=True)

In [ ]:
meals_chain.batch([{"country" :"Colombia"},
            {"country" :"Argentina"},
            {"country" :"Brasil"}])

In [ ]:
async for s in meals_chain.astream({"country": "Colombia"}):
    print(s.content, end="", flush=True)

In [ ]:
await meals_chain.ainvoke({"country":"Argentina"})

In [ ]:
await meals_chain.abatch([{"country" :"Colombia"},
            {"country" :"Argentina"},
            {"country" :"Brasil"}])

In [ ]:
"""
#CHAIN
meals_chain = meals_prompt | llm

#Sync
--------------------------------------------------------------------
# Invoke
--------------------------------------------------------------------
meals_chain.invoke({"country":"colombia"})
--------------------------------------------------------------------
# Stream
for s in meals_chain.stream({"country": "Colombia"}):
    print(s.content, end="", flush=True)
--------------------------------------------------------------------
# Batch
meals_chain.batch([{"country" :"Colombia"},
            {"country" :"Argentina"},
            {"country" :"Brasil"}])
--------------------------------------------------------------------
"""

In [ ]:
await meals_chain.ainvoke({"country":"colombia"})

In [ ]:
async for s in meals_chain.astream({"country": "Colombia"}):
    print(s.content, end="", flush=True)

In [ ]:
await meals_chain.abatch([{"country" :"Colombia"},
            {"country" :"Argentina"},
            {"country" :"Brasil"}])

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_toolkits import GmailToolkit
from langchain.agents import AgentExecutor
from langchain.pydantic_v1 import BaseModel
from langchain.memory import ConversationBufferMemory

llm=ChatOpenAI(temperature=0,model_name="gpt-4")

db = SQLDatabase.from_uri("sqlite:///langchain-demo/resources/users.db")
db_toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))


tools = db_toolkit.get_tools()

agent = initialize_agent(
    llm=llm,
    tools=tools,
    verbose=False,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
)

class AgentInputs(BaseModel):
    input: str

agent_executor = agent.with_types(input_type=AgentInputs) 

In [ ]:
agent